In [347]:
lei = extrai_lei(ps)
titulo = extrai_titulo(lei["I"])
for artigo_id, artigo_p in titulo["artigos"].items():
    
    artigo = extrai_artigo(artigo_p)
    
    print("{}(...)\n".format(artigo["texto"][:50]))
    if artigo["link"]:

        print("{}\n".format(artigo["link"]))
    
    for paragrafo_id, paragrafo_p in artigo["paragrafos"].items():
        
        paragrafo = extrai_paragrafo(paragrafo_p)
        
        print("\t{}(...)\n".format(paragrafo["texto"][:50]))
        if paragrafo["link"]:
            
            print("\t{}\n".format(paragrafo["link"]))

Art. 1º - Esta Consolidação estatui as normas que (...)

Art. 2º - Considera-se empregador a empresa, indiv(...)

	§ 1º - Equiparam-se ao empregador, para os efeitos(...)

	§ 2º - Sempre que uma ou mais empresas, tendo, emb(...)

Art. 3º - Considera-se empregado toda pessoa físic(...)

	Parágrafo único - Não haverá distinções relativas (...)

Art. 4º - Considera-se como de serviço efetivo o p(...)

	Parágrafo único - Computar-se-ão, na contagem de t(...)

	http://www.planalto.gov.br/ccivil_03/LEIS/L4072.htm#art4p

Art. 5º - A todo trabalho de igual valor correspon(...)

Art. 6o Não se distingue entre o trabalho realizad(...)

http://www.planalto.gov.br/ccivil_03/_Ato2011-2014/2011/Lei/L12551.htm#art1

	Parágrafo único.  Os meios telemáticos e informati(...)

	http://www.planalto.gov.br/ccivil_03/_Ato2011-2014/2011/Lei/L12551.htm#art1

Art. 7º Os preceitos constantes da presente Consol(...)

http://www.planalto.gov.br/ccivil_03/decreto-lei/Del8079.htm#art7

Art. 8º - As autoridades admi

In [109]:
# biblioteca para requisição http
import requests
# biblioteca para parse do HTML retornado
from bs4 import BeautifulSoup
# biblioteca de expressão regular
import re
# biblioteca utilizada para construir as urls absolutas a partir das relativas
from urllib.parse import urljoin
from collections import OrderedDict

In [8]:
# Requisição sem User-Agent não estava sendo respondida...
# http://stackoverflow.com/a/27652558/3662965
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
url = "http://www.planalto.gov.br/ccivil_03/decreto-lei/Del5452.htm"

r = requests.get(url, headers=headers)

html = r.text

lei_parsed = BeautifulSoup(html, "lxml")

http://www2.camara.leg.br/documentos-e-pesquisa/publicacoes/estnottec/copy_of_portas-abertas-1/Palestra%208.pdf

# É a expressão regular

# CONSOLIDAÇÃO DAS LEIS DO TRABALHO

In [3]:
class PedacoDeLei:
    
    def __init__(self, text, old=None):
        
        self.texto = texto
        self.old = old
        
    def __str__(self):
        
        return "{}\n{}".format(self.texto, self.old)

class Alinea(PedacoDeLei):
    
    pass

class Paragrafo(PedacoDeLei):
    
    pass

class Artigo(PedacoDeLei):
    
    pass

class Lei(PedacoDeLei):
    
    def __init__(self, titulo, texto, old=None):
        
        PedacoDeLei.__init__(self, texto, old)
        self.titulo = titulo
        
    def __str__(self):
        
        return "{}\n{}".format(self.titulo, super(Lei, self))

TODO: Parágrafo único alterado

In [333]:
lei = extrai_lei(ps)
titulo = extrai_titulo(lei["I"])
artigo = extrai_artigo(titulo["artigos"]["7"])
artigo["incisos"]["e"]
#paragrafo = extrai_paragrafo(artigo["paragrafos"]["2"]); paragrafo
#paragrafo = extrai_paragrafo(artigo["paragrafos"]["Parágrafo único"]); paragrafo
#paragrafo = extrai_paragrafo(artigo["old"]); paragrafo


[<p align="justify" style="text-indent: 35px">
 <font face="Arial" size="2"><a name="art7e"></a><strike>e) aos empregados das
 empresas de propriedade da União Federal, quando por esta
 ou pelos Estados administradas, salvo em se tratando daquelas cuja propriedade ou
 administração resultem de circunstâncias transitórias.</strike></font></p>,
 <p align="justify" style="text-indent: 35px"><font face="Arial" size="2">
 <strike><a name="art7p"></a>Parágrafo único -  Aos
 trabalhadores ao serviço de empresas industriais da União, dos Estados e dos
 Municípios, salvo aqueles classificados como funcionários públicos, aplicam-se os
 preceitos da presente Consolidação. <a href="Del8079.htm#art7c">(Incluído pelo
 Decreto-lei nº 8.079, 11.10.1945)</a> </strike>        <a href="1937-1946/Del8249.htm#art5">(Revogado
 pelo Decreto-lei nº 8.249, de 1945)</a></font></p>]

# Versão atual

In [240]:
rn_re = re.compile(r"[\n\r\t\xa0]+")

In [227]:
def particiona_por(iterator, is_novo_item):
    
    particoes = OrderedDict()
    current_item_id = None
    
    for i in iterator:
        
        item_id = is_novo_item(i)
        
        if item_id:
            current_item_id = item_id
        
        if current_item_id in particoes:
            
            particoes[current_item_id].append(i)
        elif current_item_id:
            
            particoes[current_item_id] = [i]
        
    return particoes

#### Titulos

In [176]:
titulo_re = re.compile(r"^[\n\r\t\xa0]*TÍTULO\ (?P<num>[IVX]+)$")

def is_inicio_titulo(p):
    
    match = titulo_re.match(p.text)
    
    if match:
        
        return match.group("num")
    else:
        
        return None

def extrai_lei(ps):
    
    return particiona_por(ps, is_inicio_titulo)

#### Artigos

In [266]:
artigo_re = re.compile(r"^[\n\r\t\xa0]*Art.[\n\r\t\xa0\ ]+(?P<num>\d+)")

def is_inicio_artigo(p):
    
    match = artigo_re.match(p.text)
    
    if match:
        
        return match.group("num")
    else:
        
        return None

def extrai_titulo(ps):
    
    p_titulo = ps[0]
    
    artigos = particiona_por(ps[1:], is_inicio_artigo)
    
    titulo = {"titulo": p_titulo,
              "artigos": artigos}
    
    return titulo

#### Parágrafos

In [331]:
paragrafo_re = re.compile(r"^[\n\r\t\xa0]*(§\s+(?P<num>\d+)|Parágrafo\ único)")

def is_inicio_paragrafo(p):
    
    tem_strike = p.find("strike")
    match = paragrafo_re.match(p.text)
    
    if match and not tem_strike:
        tem_num = match.group("num")
        
        if tem_num:
            return tem_num
        else:
            return "Parágrafo único"
    else:
        
        return None
    
inciso_re = re.compile(r"[\n\r\t\xa0]*(?P<num>(?:[IXV]+\ \-)|(?:[a-z]+(?=\))))")

def is_inicio_inciso(p):
    
    match = inciso_re.match(p.text)
    
    if match:
        
        return match.group("num")
    else:
        
        return None

def extrai_artigo(ps):
    
    old = []
    p = ps[0]
    
    while p.find("strike"):
        
        old.append(p)
        p = ps[len(old)]
    
    texto_p = ps[len(old)]
    
    link = None
    anchors = texto_p.find_all("a")
    if anchors and anchors[-1].has_attr("href"):
        
        relative_url = anchors[-1]["href"]
        link = urljoin(url, relative_url)
        
    paragrafos = particiona_por(ps[len(old)+1:], is_inicio_paragrafo)
    
    incisos = None
    if not paragrafos:
        
        incisos = particiona_por(ps[len(old)+1:], is_inicio_inciso)
    
    artigo = {"texto":  rn_re.sub(" ", texto_p.text).strip(),
              "incisos": incisos,
              "paragrafos": paragrafos,
              "old": old,
              "link": link}
    
    return artigo

#### Incisos

In [182]:
#TODO
# identificar unidades de texto(parágrafo, inciso, artigo etc) e 
#    trabalhar melhor a identificação de versões antigas

# talvez processar primeiro o atual
# depois extrair o antigo
# e ir preenchendo o atual com o antigo
# pq, por exemplo, Art. 15 foi alterado 2 vezes
# a estrutura atual não comporta isso

clt = []

titulo_re = re.compile(r"^TÍTULO .*$")
rn_re = re.compile(r"[\n\r\t\xa0]+")
artigo_re = re.compile(r"^Art. \d+")
paragrafo_re = re.compile(r"^(§|Parágrafo\ único)")

titulo_atual = None
artigo_atual = None
antigo = None

for p in ps:
    
    p_content = rn_re.sub(" ", p.text).strip()

    if p.find("strike"):

        antigo = p_content
        continue

    if titulo_re.match(p_content):

        titulo_atual = {"nome": p_content, 
                        "descricao": None,
                        "artigos": []}
        clt.append(titulo_atual)
    elif titulo_atual:

        if p_content.isupper():

            titulo_atual["descricao"] = p_content
        else:
            
            last_url = None
            anchors = p.find_all("a")
            if anchors and anchors[-1].has_attr("href"):
                
                relative_url = anchors[-1]['href']
                last_url = urljoin(url, relative_url)

            if artigo_re.match(p_content):

                artigo_atual = {"texto": p_content,
                                "paragrafos": [],
                                "old": antigo,
                                "link": last_url,
                                "alineas": []}
                titulo_atual["artigos"].append(artigo_atual)

                antigo = None
            elif paragrafo_re.match(p_content):
                
                paragrafo = {"texto": p_content,
                             "link": last_url,
                             "old": antigo}

                artigo_atual["paragrafos"].append(paragrafo)
                
                antigo = None
            else:
                # suponho ser continuação do parágrafo anterior
                
                alinea = {"texto": p_content,
                          "link": last_url,
                          "old": antigo}
                
                artigo_atual["alineas"].append(alinea)
                
                antigo = None

In [217]:
def print_artigo(titulo, artigo):

    artigo = clt[titulo-1]["artigos"][artigo-1]

    print("Versão antiga:\n")
    print(artigo["old"])
    print("\n")
    print("Versão atual:\n")
    print(artigo['texto'])
    print(artigo["link"])
    print("\n")
    for paragrafo in artigo["paragrafos"]:

        print("\t"+paragrafo["texto"])
        print("\n")
        
    for alinea in artigo["alineas"]:
            
        print("\t" + alinea["texto"])
        print(alinea["link"])
        print("\n")

In [218]:
print_artigo(titulo=1, artigo=7)

Versão antiga:

Art. 7º - Os preceitos constantes da presente Consolidação, salvo quando for, em cada caso, expressamente determinado em contrário, não se aplicam:


Versão atual:

Art. 7º Os preceitos constantes da presente Consolidação salvo quando fôr em cada caso, expressamente determinado em contrário, não se aplicam : (Redação dada pelo Decreto-lei nº 8.079, 11.10.1945)
http://www.planalto.gov.br/ccivil_03/decreto-lei/Del8079.htm#art7


	a) aos empregados domésticos, assim considerados, de um modo geral, os que prestam serviços de natureza não-econômica à pessoa ou à família, no âmbito residencial destas;
None


	b) aos trabalhadores rurais, assim considerados aqueles que, exercendo funções diretamente ligadas à agricultura e à pecuária, não sejam empregados em atividades que, pelos métodos de execução dos respectivos trabalhos ou pela finalidade de suas operações, se classifiquem como industriais ou comerciais;
None


	c) aos funcionários públicos da União, dos Estados e dos Mun